In [ ]:
'''
dataset: xor dataset kaggle 1000 xor pairs and their values (0 or 1)
train data: 1000

no framework (only numpy) - model: NN
init: he initialization
layer: 4 relu neurons | 2 softmax neurons
learning rate: 0.1
(cross entropy loss)
Hard code, need to improve

result: work most the time (took average 80-200 ite to get 100% acc base on init params)
ERROR: when loss ~0.69, the learning speed decrease and stop learning at 0.6928, at this specific point dw, db get too small to update

testing: print all params for all 4 cases
'''
import numpy as np

In [ ]:
load_data = np.loadtxt("xor.csv", delimiter=",", skiprows=1)
data = load_data[:1000]

train = data.T

X_train = train[:2]
Y_train = train[2:3]

In [ ]:
def init_params():
    w1 = np.random.rand(4, 2) * np.sqrt(2/2)
    b1 = np.zeros((4, 1))
    w2 = np.random.rand(2, 4) * np.sqrt(2/4)
    b2 = np.zeros((2, 1))
    return w1, b1, w2, b2

In [ ]:
def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    stable_Z = Z - np.max(Z, axis = 0, keepdims = True)
    exp_Z = np.exp(stable_Z)
    return exp_Z / np.sum(exp_Z, axis = 0, keepdims = True)

def deriReLU(Z):
    return (Z > 0) * 1

In [ ]:
def encode(Y):
    new_Y = np.zeros((1000, 2))
    new_Y[np.arange(1000), Y.astype(int)] = 1
    new_Y = new_Y.T
    return new_Y

In [ ]:
def forward_propagation(X, w1, b1, w2, b2):
    Z1 = w1 @ X + b1
    A1 = ReLU(Z1)
    Z2 = w2 @ A1 + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def backward_propagation(X, Y, Z1, A1, Z2, A2, w2):
    dz2 = A2 - encode(Y)
    dw2 = 0.001 * dz2 @ A1.T
    db2 = 0.001 * np.sum(dz2, axis = 1, keepdims = True)
    dz1 = (w2.T @ dz2) * deriReLU(Z1)
    dw1 = 0.001 * dz1 @ X.T
    db1 = 0.001 * np.sum(dz1, axis = 1, keepdims = True)
    return dw1, db1, dw2, db2

In [ ]:
def update_params(alpha, w1, b1, w2, b2, dw1, db1, dw2, db2):
    w1 = w1 - alpha*dw1
    b1 = b1 - alpha*db1
    w2 = w2 - alpha*dw2
    b2 = b2 - alpha*db2
    return w1, b1, w2, b2

In [ ]:
def accuracy(Y, A2):
    return np.sum(np.argmax(A2, axis = 0) == Y) / 1000

def cross_entropy_loss(Y, A2):
    Y_encoded = encode(Y)
    log_A2 = np.log(A2 + 1e-8)
    loss = -np.sum(Y_encoded * log_A2) / 1000
    return loss
    
def gradient_descent(X_train, Y_train, iterations, alpha):
    w1, b1, w2, b2 = init_params()
    for iteration in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(X_train, w1, b1, w2, b2)
        dw1, db1, dw2, db2 = backward_propagation(X_train, Y_train, Z1, A1, Z2, A2, w2)
        w1, b1, w2, b2 = update_params(alpha, w1, b1, w2, b2, dw1, db1, dw2, db2)
        if (iteration % 20 == 0):
            acc = accuracy(Y_train, A2)
            loss = cross_entropy_loss(Y_train, A2)
            print(f"Iteration {iteration}: {loss:.4} loss, {acc*100}% acc")
    return w1, b1, w2, b2

w1, b1, w2, b2 = gradient_descent(X_train, Y_train, 501, 0.1)

In [ ]:
# testing area

Xtest = np.array([[0,0],[0,1],[1,0],[1,1]]).T

z1,a1,z2,a2 = forward_propagation(Xtest,w1,b1,w2,b2)
print(f"z1 = {z1}\na1 = {a1}\nz2 = {z2}\na2 = {a2}")

In [ ]:
def testencode(Y):
    new_Y = np.zeros((4, 2))
    new_Y[np.arange(4), Y.astype(int)] = 1
    new_Y = new_Y.T
    return new_Y

def testbackward_propagation(X, Y, Z1, A1, Z2, A2, w2):
    dz2 = A2 - testencode(Y)
    dw2 = 0.25 * dz2 @ A1.T
    db2 = 0.25 * np.sum(dz2, axis = 1, keepdims = True)
    dz1 = (w2.T @ dz2) * deriReLU(Z1)
    dw1 = 0.25 * dz1 @ X.T
    db1 = 0.25 * np.sum(dz1, axis = 1, keepdims = True)
    return dw1, db1, dw2, db2

Ytest = np.array([[0, 1, 1, 0]])
dw1,db1,dw2,db2 = testbackward_propagation(Xtest, Ytest,z1,a1,z2,a2 , w2)
print(f"dw1 = {dw1}\ndb1 = {db1}\ndw2 = {dw2}\ndb2 = {db2}")